In [55]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [56]:
def create_driver():
    options = Options()
    options.add_argument("--start-maximized")  # Start with the browser maximized
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-notifications")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")

    # Ensure "headless" mode is NOT enabled
    driver_service = Service("/Users/jacksonsorenson/Downloads/chromedriver-mac-arm64/chromedriver")  # Replace with your ChromeDriver path
    driver = webdriver.Chrome(service=driver_service, options=options)
    return driver

In [57]:
def login_to_twitter(driver, username, password):
    try:
        driver.get("https://twitter.com/login")
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "text"))
        )
        driver.find_element(By.NAME, "text").send_keys(username, Keys.RETURN)
        time.sleep(2)  # Wait for password field
        driver.find_element(By.NAME, "password").send_keys(password, Keys.RETURN)
        time.sleep(5)  # Wait for the page to load completely
        print("Logged in to Twitter successfully.")
    except Exception as e:
        print(f"Error logging in: {e}")
        driver.quit()

In [58]:
def scrape_user_tweets(username, driver):
    tweets_data = []
    try:
        twitter_url = f"https://twitter.com/{username}"
        driver.get(twitter_url)
        time.sleep(3)  # Wait for the page to load

        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            tweets = driver.find_elements(By.XPATH, '//div[@data-testid="tweet"]')
            for tweet in tweets:
                try:
                    content = tweet.text
                    if content not in tweets_data:
                        tweets_data.append(content)
                except Exception as e:
                    print(f"Error extracting tweet: {e}")

            # Scroll down
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")

            if new_height == last_height:
                break
            last_height = new_height
        
        print(f"Scraped {len(tweets_data)} tweets for {username}.")
    except Exception as e:
        print(f"Error scraping {username}: {e}")
    return tweets_data

In [59]:
def main():
    # Replace with your CSV file path
    csv_file = "/Users/jacksonsorenson/Documents/Computational Media Lab/X/Original CSV/VAMoS X-Twitter Handles _ Verified and Public.csv"
    usernames_df = pd.read_csv(csv_file)
    usernames = usernames_df['X/Twitter'].dropna()

    # Replace with your Twitter credentials
    twitter_username = "HJSORE"
    twitter_password = "Tank2626!"

    all_tweets = {}

    driver = create_driver()
    login_to_twitter(driver, twitter_username, twitter_password)

    for username in usernames:
        print(f"Scraping tweets for {username}...")
        tweets = scrape_user_tweets(username, driver)
        all_tweets[username] = tweets

    driver.quit()

    tweets_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in all_tweets.items()]))
    tweets_df.to_csv("scraped_tweets.csv", index=False)
    print("Scraped tweets saved to CSV.")

if __name__ == "__main__":
    main()

Logged in to Twitter successfully.
Scraping tweets for @0X134340_...
Scraped 0 tweets for @0X134340_.
Scraping tweets for @antoguino...
Error scraping @antoguino: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.86)
Stacktrace:
0   chromedriver                        0x0000000102d73ac4 cxxbridge1$str$ptr + 3651580
1   chromedriver                        0x0000000102d6c314 cxxbridge1$str$ptr + 3620940
2   chromedriver                        0x00000001027d44b4 cxxbridge1$string$len + 89224
3   chromedriver                        0x00000001027af994 core::str::slice_error_fail::ha0e52dbcb60e6bae + 3780
4   chromedriver                        0x000000010283e890 cxxbridge1$string$len + 524388
5   chromedriver                        0x000000010285176c cxxbridge1$string$len + 601920
6   chromedriver                        0x000000010280d0b0 cxxbridge1$string$len + 321668
7   chromedriver                        0x000